In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! nvidia-smi

# **Install Dependencies**

In [ ]:
%%capture
%pip install tensorflow
%pip install keras
%pip install netron
%pip install nltk

#%pip3 install git+https://github.com/huggingface/transformers
%pip install datasets
%pip install nlp
%pip install rouge-score
!pip install sacrebleu
%pip install git-python
%pip install sentencepiece
!pip install torch
!pip install transformers
%pip install tree-sitter
%pip install evaluate
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install sacremoses
%pip install torchdata
%pip install gradio
%pip install evaluate
%pip install tensorboard
%pip install netron
!pip install accelerate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric, Dataset
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
logging.basicConfig(level=logging.INFO)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, EncoderDecoderModel
import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


# **Clear cache**

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

In [ ]:
%pwd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/88kData.csv')
df.head(10)

In [ ]:
#remove ID column
df = df.drop('ID', axis=1)
# remove Null value
# df = df.dropna()

In [ ]:
NewList = []

In [ ]:
# append unneeded fatwa

NewList+=list(df[df['ans'].str.contains('انظر إلى الفتوى')].index)
NewList+=list(df[df['ans'].str.contains('رقم:')].index)
NewList+=list(df[df['ans'].str.contains('فتوى رقم')].index)

In [ ]:
# drop unneeded fatwa
df = df.drop(index=NewList, axis=0)

In [ ]:
df[df['ans'].str.contains('นู')]

In [ ]:
df = df.drop(index=84488, axis=0)

In [ ]:
# No. of new records
len(df)

In [ ]:
import re

In [ ]:
def extract_arabic_and_english_numerics(text):
    # remove english letter
    arabic_pattern = r'[^a-zA-Z\s0-9]+'
    numeric_pattern = r'\d+'
    combined_pattern = f'({arabic_pattern}|{numeric_pattern})'

    matches = re.findall(combined_pattern, text)
    extracted_text = ' '.join(matches)

    return extracted_text

In [ ]:
# #Applying the function on the ['ans'] column
df['ans'] = df['ans'].apply(extract_arabic_and_english_numerics)
df['ans']

In [ ]:
#Applying the function on the ['ques'] column
df['ques'] = df['ques'].apply(extract_arabic_and_english_numerics)
df['ques']

In [ ]:
newlist2 = []

In [ ]:
#Visualize the answers with length less than 118 charachter
for k in range(len(df)):

    if len(df['ques'].iloc[k]) < 118:
        print(df['ques'].iloc[k])

In [ ]:
#Dropping every row that contains an answer with length less than 118 charachter
for k in range(len(df)):

    if len(df['ans'].iloc[k]) < 118:
        newlist2.append(k)

In [ ]:
# remove from data
newlist2 = list(df.iloc[newlist2].index)
df = df.drop(index=newlist2, axis=0)

In [ ]:
#Making all the dataset columns as a string, and dropping duplicates from the dataset.
df = df.astype(str)
df.drop_duplicates(inplace=True)

In [ ]:
# resize data to 20000
df = df[0:20000]

In [ ]:
#Save the cleaned version of the data to the Machine as .csv file
df.to_csv("Finalized_Data2.csv",index=False)

In [ ]:
df = pd.read_csv("Finalized_Data2.csv")

In [ ]:
df.head()

In [ ]:
df["full_text"]=" question: " + df["title"]+df["ques"]+" answer: "+df["ans"]

In [ ]:
df["full_text"][2]

# **Fetch dataset**

In [ ]:
# !wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

In [ ]:
# all_data = load_dataset("/content/Arabic-Empathetic-Chatbot/model/ArabicEmpatheticDialogues.py")


all_data = load_dataset('csv', data_files={"/content/Finalized_Data2.csv"},delimiter=',', quotechar= '"')

train_dataset = all_data['train'].train_test_split(test_size=0.2,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.2,seed=42)['test']
val_dataset = val_data.train_test_split(test_size=0.5,seed=42)['train']
test_dataset = val_data.train_test_split(test_size=0.5,seed=42)['test']


print("Length of train data",len(train_dataset))
print("Length of dev data",len(val_dataset))
print("Length of test data",len(test_dataset))

In [ ]:
from prettytable import PrettyTable

# Select the first 5 records from the dataset
subset = train_dataset.select(range(15))

# Define the table headers
table = PrettyTable()
table.field_names = ['ques', "ans"]

# Add each record to the table
for i in range(len(subset)):
    table.add_row([
        # subset[i]["title"],
        subset[i]['ques'],
         subset[i]['ans']
        # subset[i]["full_text"]
    ])


# Print the table
print(table)





In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
test_dataset

# **Load Model and Tokenizer**

In [ ]:

model_name = "asafaya/bert-base-arabic"
model_name2 = "akhooli/gpt2-small-arabic"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [ ]:

#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name2)
#model.to("cuda")


In [ ]:
model

In [ ]:
#set special tokens
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params
model.config.max_length = 128
model.config.early_stopping = True

model.config.num_beams = 1
model.config.vocab_size = model.config.encoder.vocab_size


In [ ]:
model.base_model.config.encoder.num_hidden_layers = 6
model.base_model.config.decoder.num_hidden_layers = 6

In [ ]:
model.tie_weights = True

# **Save model**

In [ ]:
model.save_pretrained("bert2gpt-model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("./bert2gpt-model")
model

# **Evaluator**

In [ ]:
from sacrebleu import corpus_bleu
def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}





# **Prepare the data**

In [ ]:
model.config.max_length

In [ ]:


from transformers import BertConfig, GPT2Config, BertModel, GPT2Model

# Load the BERT configuration
bert_config = BertConfig.from_pretrained("bert-base-uncased")

# Load the GPT2 configuration
gpt2_config = GPT2Config.from_pretrained("gpt2")

# Set the decoder_start_token_id attribute in the GPT2 configuration
gpt2_config.decoder_start_token_id = "<cls>"

# Load the BERT and GPT2 models separately
bert_model = BertModel.from_pretrained("bert-base-uncased", config=bert_config)
gpt2_model = GPT2Model.from_pretrained("gpt2", config=gpt2_config)

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 8


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["Question"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["Answer"], padding="max_length", truncation=True, max_length=decoder_length)

    #input_ids = inputs.input_ids.to("cuda")
    #attention_mask = inputs.attention_mask.to("cuda")

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=[ 'ques', 'ans'],)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=[ 'ques', 'ans'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=128 , padding=True,
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./bert2gpt-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    num_train_epochs=200,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
    optim = "adamw_hf",
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

In [ ]:
#Training
trainer.train()

In [ ]:
trainer.save_model()
trainer.save_state()

# **Copy to Google Drive**

In [ ]:
import shutil

# Define the source and destination folders
src_folder = '/content/bert2gpt-Training'
dst_folder = '/content/drive/MyDrive/bert2-Training'

# Copy the folder and its contents to Google Drive
shutil.copytree(src_folder, dst_folder)

# **Visualize**

In [ ]:
%load_ext tensorboard

In [ ]:
#%reload_ext tensorboard

In [ ]:
pwd

In [ ]:

tensorboard --logdir ./bert2-Training/runs/

# **Inference and Testing**

In [ ]:
#model_name = "bert2gpt-Training"
model_name = "/content/drive/MyDrive/bert2-Training"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
new_model.to("cuda")
new_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
# map data correctly
def generate_response(context):
    inputs = new_tokenizer([context], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_response = new_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return output_response

In [ ]:
input_context = 'ما حكم الخمر'
output_response = generate_response(input_context)

In [ ]:
print(output_response)

In [ ]:
import gradio as gr

iface = gr.Interface(fn=generate_response, inputs="text", outputs="text",
                     title="نَوَّرْ",
                     description="This is a solution for question answering task")
iface.launch()